In [25]:
import pandas as pd

# read the training data
df = pd.read_csv('train.csv')
# df.head()

In [26]:
authors = dict([(auth, idx) for idx, auth in enumerate(df['author'].unique())])
print(authors)
df['author_id'] = df['author'].apply(lambda x: authors[x])
# df.head()

{'HPL': 1, 'EAP': 0, 'MWS': 2}


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.text, df.author_id, test_size=0.13, random_state=42)
# X_train.head()

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(17033, 23791)

In [29]:
count_vect.vocabulary_.get(u'algorithm')

In [30]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(17033, 23791)

In [31]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [32]:
docs_new = ['I am not sure to what limit his knowledge may extend.', 'For I am Iranon, who was a Prince in Aira.']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, df.author))

'I am not sure to what limit his knowledge may extend.' => 0        EAP
1        HPL
2        EAP
3        MWS
4        HPL
5        MWS
6        EAP
7        EAP
8        EAP
9        MWS
10       MWS
11       EAP
12       HPL
13       HPL
14       EAP
15       MWS
16       EAP
17       MWS
18       EAP
19       HPL
20       EAP
21       HPL
22       EAP
23       EAP
24       EAP
25       EAP
26       EAP
27       EAP
28       HPL
29       HPL
        ... 
19549    MWS
19550    EAP
19551    EAP
19552    EAP
19553    EAP
19554    HPL
19555    EAP
19556    EAP
19557    EAP
19558    EAP
19559    HPL
19560    EAP
19561    HPL
19562    EAP
19563    MWS
19564    EAP
19565    EAP
19566    MWS
19567    EAP
19568    EAP
19569    MWS
19570    MWS
19571    HPL
19572    EAP
19573    MWS
19574    EAP
19575    EAP
19576    EAP
19577    EAP
19578    HPL
Name: author, Length: 19579, dtype: object
'For I am Iranon, who was a Prince in Aira.' => 0        EAP
1        HPL
2        EAP
3        MWS
4    

In [33]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [34]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [35]:
import numpy as np
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test) 

0.8142183817753339

In [1]:
from sklearn.linear_model import SGDClassifier

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None)),
])

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test) 

NameError: name 'Pipeline' is not defined

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted,
    target_names=df.author))

In [ ]:
metrics.confusion_matrix(y_test, predicted)

In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

models = [('MultiNB', MultinomialNB(alpha=0.03)),
          ('Calibrated MultiNB', CalibratedClassifierCV(
              MultinomialNB(alpha=0.03), method='isotonic')),
          ('Calibrated BernoulliNB', CalibratedClassifierCV(
              BernoulliNB(alpha=0.03), method='isotonic')),
          ('Calibrated Huber', CalibratedClassifierCV(
              SGDClassifier(loss='modified_huber', alpha=1e-4,
                            max_iter=10000, tol=1e-4), method='sigmoid')),
          ('Logit', LogisticRegression(C=30))]

train = pd.read_csv('train.csv')
vectorizer=TfidfVectorizer(token_pattern=r'\w{1,}', sublinear_tf=True, ngram_range=(1,2))
clf = VotingClassifier(models, voting='soft', weights=[3,3,3,1,1])
X_train = vectorizer.fit_transform(train.text.values)
authors = ['MWS','EAP','HPL']
y_train = train.author.apply(authors.index).values
clf.fit(X_train, y_train)

test = pd.read_csv('test.csv', index_col=0)
X_test = vectorizer.transform(test.text.values)
results = clf.predict_proba(X_test)
pd.DataFrame(results, index=test.index, columns=authors).to_csv('results.csv')

In [40]:
from collections import defaultdict

#import keras
#import keras.backend as K
from keras.layers import Dense, GlobalAveragePooling1D, Embedding
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

np.random.seed(7)



AttributeError: module 'pandas' has no attribute 'computation'

In [2]:

from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

Using TensorFlow backend.
